In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Colon_and_Rectal_Cancer"
cohort = "GSE46517"

# Input paths
in_trait_dir = "../../input/GEO/Colon_and_Rectal_Cancer"
in_cohort_dir = "../../input/GEO/Colon_and_Rectal_Cancer/GSE46517"

# Output paths
out_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/GSE46517.csv"
out_gene_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/gene_data/GSE46517.csv"
out_clinical_data_file = "../../output/preprocess/Colon_and_Rectal_Cancer/clinical_data/GSE46517.csv"
json_path = "../../output/preprocess/Colon_and_Rectal_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Human melanoma samples comparing nevi and primary and metastatic melanoma"
!Series_summary	"We sought to identify genes and gene signatures which correlate with progression by sampling human melanomas from nevi, primary, and metastatic tumors. The large number of samples also permits analysis within groups."
!Series_overall_design	"Human melanoma samples were isolated from historical frozen patient specimens. RNA was extracted and run on the human Affymetrix U133A microarray chip."
Sample Characteristics Dictionary:
{0: ['tissue type: Metastatic Melanoma', 'tissue type: Primary Melanoma', 'tissue type: Nevus', 'tissue type: Normal Skin', 'tissue type: Normal Epithelial Melanocytes'], 1: [nan, 'patient id: 35', 'patient id: 23', 'patient id: 13', 'patient id: 6', 'patient id: 40', 'patient id: 26', 'patient id: 1', 'patient id: 22', 'patient id: 4', 'patient id: 30', 'patient id: 5', 'patient id: 15', 'patient id: 20', 'patient id: 17', 'patient id

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# The background information mentions Affymetrix U133A microarray chip, which is used for gene expression analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait data (Melanoma vs. other tissue types)
trait_row = 0  # "tissue type" contains information about the Melanoma status

# For age data
age_row = 7  # "age at time of resection" contains age information

# For gender data
gender_row = 8  # "gender" contains gender information

# 2.2 Data Type Conversion

# Convert trait data (Melanoma vs. normal)
def convert_trait(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Binary classification: 1 for melanoma (primary or metastatic), 0 for non-melanoma (nevus, normal skin, normal melanocytes)
    if "Metastatic Melanoma" in value or "Primary Melanoma" in value:
        return 1
    elif "Nevus" in value or "Normal" in value:
        return 0
    return None

# Convert age data to numeric values
def convert_age(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Extract age in years from format like "72y 4m"
    if "y" in value:
        try:
            years = float(value.split("y")[0])
            return years
        except ValueError:
            return None
    return None

# Convert gender data to binary (0 for female, 1 for male)
def convert_gender(value):
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "female":
        return 0
    elif value == "male":
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save metadata using the provided function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Note: This dataset appears to be a melanoma dataset, not Colon_and_Rectal_Cancer.
# We'll proceed with initial validation and skip clinical feature extraction
# since the expected clinical_data.csv file does not exist.

print(f"Initial validation complete. Trait availability: {is_trait_available}, Gene availability: {is_gene_available}")
print(f"Note: This appears to be a melanoma dataset rather than {trait}.")


Initial validation complete. Trait availability: True, Gene availability: True
Note: This appears to be a melanoma dataset rather than Colon_and_Rectal_Cancer.


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 92
Header line: "ID_REF"	"GSM1131566"	"GSM1131567"	"GSM1131568"	"GSM1131569"	"GSM1131570"	"GSM1131571"	"GSM1131572"	"GSM1131573"	"GSM1131574"	"GSM1131575"	"GSM1131576"	"GSM1131577"	"GSM1131578"	"GSM1131579"	"GSM1131580"	"GSM1131581"	"GSM1131582"	"GSM1131583"	"GSM1131584"	"GSM1131585"	"GSM1131586"	"GSM1131587"	"GSM1131588"	"GSM1131589"	"GSM1131590"	"GSM1131591"	"GSM1131592"	"GSM1131593"	"GSM1131594"	"GSM1131595"	"GSM1131596"	"GSM1131597"	"GSM1131598"	"GSM1131599"	"GSM1131600"	"GSM1131601"	"GSM1131602"	"GSM1131603"	"GSM1131604"	"GSM1131605"	"GSM1131606"	"GSM1131607"	"GSM1131608"	"GSM1131609"	"GSM1131610"	"GSM1131611"	"GSM1131612"	"GSM1131613"	"GSM1131614"	"GSM1131615"	"GSM1131616"	"GSM1131617"	"GSM1131618"	"GSM1131619"	"GSM1131620"	"GSM1131621"	"GSM1131622"	"GSM1131623"	"GSM1131624"	"GSM1131625"	"GSM1131626"	"GSM1131627"	"GSM1131628"	"GSM1131629"	"GSM1131630"	"GSM1131631"	"GSM1131632"	"GSM1131633"	"GSM1131634"	"GSM1131635"	"GSM1131636"	"GSM1131637"	"GSM1131638"	

Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1598_g_at', '160020_at', '1729_at', '1773_at', '177_at',
       '179_at', '1861_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers provided in the data, these appear to be probe IDs
# rather than standard human gene symbols. Specifically, these look like Affymetrix
# probe identifiers (e.g., "1007_s_at", "1053_at") which need to be mapped to 
# official gene symbols.

# Affymetrix probe IDs typically have formats like "####_at", "####_s_at", etc.
# and require mapping to convert to standard gene symbols (like BRCA1, TP53, etc.)

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns in the gene annotation dataframe that contain probe IDs and gene symbols
probe_col = 'ID'  # Column containing probe IDs like "1007_s_at"
gene_col = 'Gene Symbol'  # Column containing gene symbols like "DDR1 /// MIR4640"

# 2. Get a mapping dataframe using the function from the library
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the shape of the resulting gene expression dataframe and preview a few rows
print(f"Gene data shape: {gene_data.shape}")
print("First few genes with their expression values:")
print(gene_data.head())


Gene data shape: (13830, 121)
First few genes with their expression values:
        GSM1131566   GSM1131567   GSM1131568   GSM1131569  GSM1131570  \
Gene                                                                    
A1CF    144.908113   119.080971   114.282289   118.952864  118.819808   
A2M     445.552804  1715.715807  3180.960949  2705.682868  680.621470   
A4GALT   93.274356    60.266460    31.331144    75.625633   50.981658   
A4GNT   201.909554   196.655163   133.220356   276.103770  144.675771   
AAAS    200.259891   203.574159   155.812455   203.366706  175.886595   

        GSM1131571   GSM1131572  GSM1131573  GSM1131574  GSM1131575  ...  \
Gene                                                                 ...   
A1CF    110.742702   138.952519  156.039543  131.423745  175.564547  ...   
A2M     970.836927  1145.824529  793.454716  223.795857  369.923699  ...   
A4GALT   22.568745   136.231870  131.918490   22.343044   91.924361  ...   
A4GNT   160.380270   187.058991 

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the functions defined in step 2
# First, let's load the clinical data again to ensure we have the latest version
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features using melanoma vs normal tissue as the binary trait
selected_clinical_df = geo_select_clinical_features(
    clinical_data, 
    trait="Melanoma", 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 3. Link the clinical and genetic data
# Transpose normalized gene data for linking
gene_data_t = normalized_gene_data.T

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, "Melanoma")
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, "Melanoma")

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data comparing melanoma (primary and metastatic) with normal tissue/nevi."
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (13542, 121)
First few genes with their expression values after normalization:
        GSM1131566   GSM1131567   GSM1131568   GSM1131569  GSM1131570  \
Gene                                                                    
A1CF    144.908113   119.080971   114.282289   118.952864  118.819808   
A2M     445.552804  1715.715807  3180.960949  2705.682868  680.621470   
A4GALT   93.274356    60.266460    31.331144    75.625633   50.981658   
A4GNT   201.909554   196.655163   133.220356   276.103770  144.675771   
AAAS    200.259891   203.574159   155.812455   203.366706  175.886595   

        GSM1131571   GSM1131572  GSM1131573  GSM1131574  GSM1131575  ...  \
Gene                                                                 ...   
A1CF    110.742702   138.952519  156.039543  131.423745  175.564547  ...   
A2M     970.836927  1145.824529  793.454716  223.795857  369.923699  ...   
A4GALT   22.568745   136.231870  131.918490   22.343044   91.924361  ...   
A

Normalized gene data saved to ../../output/preprocess/Colon_and_Rectal_Cancer/gene_data/GSE46517.csv
Clinical data saved to ../../output/preprocess/Colon_and_Rectal_Cancer/clinical_data/GSE46517.csv
Clinical data preview:
{'GSM1131566': [1.0, nan, nan], 'GSM1131567': [1.0, nan, nan], 'GSM1131568': [1.0, nan, nan], 'GSM1131569': [1.0, nan, nan], 'GSM1131570': [1.0, nan, nan], 'GSM1131571': [1.0, nan, nan], 'GSM1131572': [1.0, nan, nan], 'GSM1131573': [1.0, nan, nan], 'GSM1131574': [1.0, nan, nan], 'GSM1131575': [1.0, nan, nan], 'GSM1131576': [1.0, nan, nan], 'GSM1131577': [1.0, nan, nan], 'GSM1131578': [1.0, nan, nan], 'GSM1131579': [1.0, nan, nan], 'GSM1131580': [1.0, nan, nan], 'GSM1131581': [1.0, nan, nan], 'GSM1131582': [1.0, nan, nan], 'GSM1131583': [1.0, nan, nan], 'GSM1131584': [1.0, nan, nan], 'GSM1131585': [1.0, nan, nan], 'GSM1131586': [1.0, nan, nan], 'GSM1131587': [1.0, nan, nan], 'GSM1131588': [1.0, nan, nan], 'GSM1131589': [1.0, nan, nan], 'GSM1131590': [1.0, nan, nan], 'G

Data after handling missing values: (121, 13545)
For the feature 'Melanoma', the least common label is '0.0' with 17 occurrences. This represents 14.05% of the dataset.
The distribution of the feature 'Melanoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 61.645161290322584
  50% (Median): 61.645161290322584
  75%: 61.645161290322584
Min: 29.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 14 occurrences. This represents 11.57% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Colon_and_Rectal_Cancer/cohort_info.json


Linked data saved to ../../output/preprocess/Colon_and_Rectal_Cancer/GSE46517.csv
